![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use Watsonx to analyze car rental customer satisfaction and offer recommendation.

**Note:** Please note that for the watsonx challenge, please run these notebooks in IBM Cloud and not on on your laptop/desktop.

This notebook contains the steps and code to demonstrate support of text sentiment analysis in Watsonx. It introduces commands for data retrieval, model testing and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.10.

<a id="setup"></a>
## Set up the environment

### Install and import the dependecies

In [ ]:
# !pip install datasets | tail -n 1
# !pip install scikit-learn | tail -n 1
# !pip install ibm-watson-machine-learning==1.0.349 | tail -n 1

**Note:** Please restart the notebook kernel to pick up proper version of packages installed above.

### Setup .env file

In [ ]:
!echo "PROJECT_ID=projectid" > .env
!echo "apikey=my-api-key" >> .env

In [1]:
import os
from dotenv import dotenv_values
from pandas import read_csv

In [37]:
config=dotenv_values()
print(config.keys())

odict_keys(['PROJECT_ID', 'apikey'])


### Watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. Instructions have been provided to generate IBM Cloud API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [128]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = config["apikey"],
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. When you run notebook on IBM Cloud, project in which it runs is saved as environment variable PROJECT_ID.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [4]:
try:
    project_id = config["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Train/test data loading

Load train and test datasets. At first, training dataset (`train_data`) should be used to work with the models to prepare and tune prompt. Then, test dataset (`test_data`) should be used to calculate the metrics score for selected model, defined prompts and parameters.

In [5]:
filename_test = 'data/test.csv'
filename_train = 'data/train.csv'

test_data = read_csv(filename_test)
train_data = read_csv(filename_train)

In [6]:
train_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2944,Female,M,2,41.92,Active,No,Customer service was friendly and helpful.,1,Service: Knowledge,NaN
1,1119,Female,M,2,33.60,Active,Yes,Customer service was good at MSP airport and t...,1,Service: Knowledge,NaN
2,0,Male,M,0,51.00,Inactive,Yes,I do not understand why I have to pay additio...,0,Product: Pricing and Billing,Premium features
3,1085,Female,S,2,42.00,Inactive,No,Based on the customer service personnel I enco...,0,Service: Attitude,On-demand pickup location
4,0,Female,M,2,44.10,Active,No,Provide more convenient car pickup from the ai...,0,Service: Orders/Contracts,On-demand pickup location


In [7]:
test_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2771,Female,M,2,49.99,Inactive,No,"last time I rented a car was at Manchester, NH...",0,Product: Functioning,On-demand pickup location
1,1133,Male,S,1,56.05,Inactive,No,Please lower the prices.,0,Product: Pricing and Billing,Free Upgrade
2,900,Female,M,1,64.64,Active,No,Excellent response dealing with child seat.,1,Service: Accessibility,NaN
3,3795,Male,M,0,46.51,Inactive,No,"all went quite smoothly... it was Enterprise, ...",1,Service: Accessibility,NaN
4,3541,Male,S,1,17.01,Inactive,Yes,"Slow, long lineup",0,Product: Functioning,On-demand pickup location


<a id="models"></a>
## Foundation Models on Watsonx



Below code invokes Watson Machine Learning API to invoke Watsonx.ai LLMs


In [8]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

<a id="predict"></a>
## Evaluate the model, prompt and parameters

### **1. Customer satisfaction**

Define instructions for the model to recognize if customer was satisfied or unsatisfied.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [25]:
satisfaction_instruction = """

You are a sentiment analysis expert. Analyze the following customer feedback and determine if the customer is satisfied or unsatisfied. Output 0, if the customer is unsatisfied and 1 if the customer is satisfied. Consider the context, tone, and any nuanced expressions. Here are some examples to guide you:

Customer service was friendly and helpful.

1

Customer service was good at MSP airport and the process was very fast.  From getting off of the plane to leaving with my rental car was less than 45 minutes.

1

I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.

0

Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.

0

Provide more convenient car pickup from the airport parking.

0

all went quite without hassle, they picked me up to get the car it was great.

1

We got our car very quickly.

1

they should not try so hard to up sell

0

They were friendly and helpful enough.

1

The customer service people were very nice.  During my vacation I locked my keys in the car.  The rental company sent someone out with a new set of keys to unlock the car.

1

"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [31]:
parameters = {
    "decoding_method": "greedy",""
    "max_new_tokens": 10,
    "min_new_tokens": 1,
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v2"

Analyze the customer satisfaction for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [32]:
results = []
prompt = Prompt(access_token, project_id)
comments = list(test_data.Customer_Service)
satisfaction = list(test_data.Satisfaction.astype(str))

for input_text in comments:
    prompt_input = satisfaction_instruction + "\n" + input_text + "\n"
    results.append(prompt.generate(prompt_input, model_id, parameters).replace("\n",""))

### Calculate the F1 micro score

In [33]:
results

['0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1']

In [35]:
from sklearn.metrics import f1_score

print('Sentiment analysis f1_micro_score', f1_score(satisfaction, results, average='micro'))

Sentiment analysis f1_micro_score 0.92


### **2. Offer Recommendation**

Define instructions for the model to recommend best offer to an unsatisfied customer.

**Note:** Please **start with using [watsonx.ai Prompt Lab](https://dataplatform.cloud.ibm.com/wx/home?context=wx)** to find better prompts that provides you the best result on a small subset training records (under `train_data` variable). Make sure to not run an inference of all of `train_data`, as it'll take a long time to get the results. To get a sample from `train_data`, you can use e.g.`train_data.head(n=10)` to get first 10 records, or `train_data.sample(n=10)` to get random 10 records. Only once you have identified the best performing prompt, update this notebook to use the prompt and compute the metrics on the test data.

**Action:** Please edit the below cell and add your own prompt here. In the below prompt, we have the instruction (first sentence) and one example included in the prompt.  If you want to change the prompt or add your own examples or more examples, please change the below prompt accordingly.

In [108]:
for i, row in train_data.loc[train_data["Satisfaction"] == 0][["Customer_Service","Action"]].iterrows():
    print(row["Customer_Service"])
    print(row["Action"])
    print()

I do not  understand why I have to pay additional fee if vehicle is returned without a full tank.
Premium features

Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance.
On-demand pickup location

Provide more convenient car pickup from the airport parking.
On-demand pickup location

VERY slow service!
Free Upgrade

They could really try work harder.
Free Upgrade

I would like the personnel to pretend they care about customer, at least.
On-demand pickup location

The customer service agent was helpful, but I find that providing all of the information to the agent takes too long.  It would be nice if I could provide all of the information to the agent prior to coming the rental place.
On-demand pickup location

I had to wait in line for a long time to get and return the vehicle.  Also, th

In [136]:
offer_recommendation_instruction = """You are a customer service assistant designed to enhance customer satisfaction. Your task is to classify customer feedback and recommend the most appropriate offer to address their dissatisfaction. The available offers are "On-demand pickup location", "Free Upgrade", "Voucher", and "Premium features". Output only one of these options and nothing else.
Classification Hints:

    On-demand pickup location: Select this if the feedback mentions issues with delivery location, convenience, or the need for flexible pickup options.
    Free Upgrade: Select this if the feedback highlights problems with the quality, condition, or availability of the product/service.
    Voucher: Select this if the feedback focuses on pricing issues or expresses dissatisfaction with the cost relative to value.
    Premium features: Select this if the feedback includes unresolved issues, ongoing problems, or dissatisfaction with the support experience.

Examples:

Customer Feedback: "I do not understand why I have to pay an additional fee if the vehicle is returned without a full tank."
Best Offer: Premium features

Customer Feedback: "Based on the customer service personnel I encountered most recently, I would say it is vastly preferable for the personnel to be able to at least pretend to care whether the customer ever actually receives a car rental that was reserved months in advance."
Best Offer: On-demand pickup location

Customer Feedback: "Provide more convenient car pickup from the airport parking."
Best Offer: On-demand pickup location

Customer Feedback: "VERY slow service!"
Best Offer: Free Upgrade

Customer Feedback: "They could really try work harder."
Best Offer: Free Upgrade

Customer Feedback: "I would like the personnel to pretend they care about customer, at least."
Best Offer: On-demand pickup location

Customer Feedback: "The customer service agent was helpful, but I find that providing all of the information to the agent takes too long. It would be nice if I could provide all of the information to the agent prior to coming the rental place."
Best Offer: On-demand pickup location

Customer Feedback: "I had to wait in line for a long time to get and return the vehicle. Also, the car was not clean."
Best Offer: Voucher

Customer Feedback: "It was absolutely ATROCIOUS! My wife and I were in a foreign country when we realized that our car had an expired license plate and expired proof of insurance!"
Best Offer: Voucher

Customer Feedback: "They should not try so hard to up sell."
Best Offer: Free Upgrade

Customer Feedback: "Long lines waiting for the rental pick."
Best Offer: On-demand pickup location

Customer Feedback: "What customer service? It was a nightmare."
Best Offer: Voucher

Customer Feedback: "Sorely lacking helpdesk was not much helpful."
Best Offer: Voucher

Customer Feedback: "Delayed shuttle, almost missed flight, bad customer service."
Best Offer: Free Upgrade

Customer Feedback: "I would like the reps be knowledgeable about the immediate area around the rental agency and or have maps for the area available free of charge."
Best Offer: Premium features

Customer Feedback: "Did not have some problems. My car was not ready on time."
Best Offer: Free Upgrade

Customer Feedback: "Initially the representative handled the situation badly. No car was available, even next day. Then the rep helped to find us a car in another agency."
Best Offer: Free Upgrade

Customer Feedback: "It would be nice if they included maps to the airport drop off with the car, last time we got lost returning the car and almost missed the flight. There was nothing on the exit that indicated rental returns."
Best Offer: Premium features

Customer Feedback: "States have different rules and regulations pertaining to speed, radar detectors, even when light should be on. Customer services reps should have a handout or be knowledgeable of a particular state's laws."
Best Offer: Premium features

Customer Feedback: "Price too high and location off premise too far."
Best Offer: On-demand pickup location

Customer Feedback: "The company was overwhelmed by the number of customers versus the number of available agents and they were not articulating their situation to the customers well enough. I think we waited for almost 3 hours just to get a rental car. It was ridiculous."
Best Offer: On-demand pickup location

Customer Feedback: "Even economy class cars are too expensive."
Best Offer: On-demand pickup location

Customer Feedback: "Customer service was great until we returned the car. I experienced an issue with damage to the car, and felt that things were left unanswered and still waiting for a response from them."
Best Offer: Premium features

Customer Feedback: "I have had a few recent rentals that have taken a very very long time, with no offer of apology. In the most recent case, the agent subsequently offered me a car type on an upgrade coupon and then told me it was no longer available because it had just been taken."
Best Offer: Voucher

Customer Feedback: "Price TOO HIGH and off premise location is NOT convenient."
Best Offer: On-demand pickup location

Customer Feedback: "Having knowledgeable personnel is very important."
Best Offer: Voucher

Customer Feedback: "Friendly, but could not make any decisions on their own."
Best Offer: Premium features

Customer Feedback: "It seems that they never have the class of car that I ordered. Also, the last time I got there earlier than I thought and they wouldn't let me have the car until the time that I had requested it because their computer wouldn't allow it."
Best Offer: Free Upgrade

Customer Feedback: "They were too pushy in trying to sell insurance."
Best Offer: Free Upgrade

Customer Feedback: "Customer Service at Alamo was RUDE. We waited in line for over an hour to get our car. Car was bad. Had to change it a few days later. Another family traveling with us was upgraded to a larger car as they were out of the type of car booked."
Best Offer: Free Upgrade

Customer Feedback: "They were idiots. The car had problems and they were unable to fix them or provide a replacement without a lot of hassle."
Best Offer: Free Upgrade

Customer Feedback: "I had to pay more because of some hidden fees. It disappointed me."
Best Offer: Free Upgrade

Customer Feedback: "They could not provide driving directions."
Best Offer: Premium features

Customer Feedback: "I thought that they were very short and not very friendly. I felt like they hated their job and could care less about the customer."
Best Offer: Voucher

Customer Feedback: "The car should have been brought to us instead of us trying to find it in the lot."
Best Offer: On-demand pickup location

Customer Feedback: "They did not have the car I wanted. They upgraded me to a car I did not like and did not want."
Best Offer: Free Upgrade

Customer Feedback: "Had problems with windshield wipers that affected usability. Problem was not solved. Customer service was NOT helpful. Because of franchising, I was not able to get help from an office other than the one I rented from. I had driven 60 miles from that location and there was another location 2 miles from where I was staying."
Best Offer: Free Upgrade

Customer Feedback: "You need more staff to accommodate travelers who are in a hurry. Specifically, cut down the wait time to either retrieve or turn in a rental car."
Best Offer: On-demand pickup location

Customer Feedback: "The only unpleasant aspects of interacting with personnel from the company was having to wait in a long line since most windows were closed. Also, they tried really hard to sell additional insurance coverage which was obnoxious."
Best Offer: Free Upgrade

Customer Feedback: "They were a little slow."
Best Offer: Premium features

Customer Feedback: "Please back to lower the prices."
Best Offer: Free Upgrade


Now it's your turn:

Customer Feedback:"""

### Defining the model parameters
We need to provide a set of model parameters that will influence the result. We will use IBM's Granite model.

In [120]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 8,
    "min_new_tokens": 1,
    "stop_sequences": ["Premium features","On-demand pickup location","Free Upgrade","Voucher"],
    "repetition_penalty": 1
}

model_id = "ibm/granite-13b-instruct-v2"

Filter test data for unsatisfied customer

In [58]:
unsatisfied_test_data = test_data.loc[test_data['Satisfaction'] == 0]
unsatisfied_test_data.head()

,ID,Gender,Status,Children,Age,Customer_Status,Car_Owner,Customer_Service,Satisfaction,Business_Area,Action
0,2771,Female,M,2,49.99,Inactive,No,"last time I rented a car was at Manchester, NH...",0,Product: Functioning,On-demand pickup location
1,1133,Male,S,1,56.05,Inactive,No,Please lower the prices.,0,Product: Pricing and Billing,Free Upgrade
4,3541,Male,S,1,17.01,Inactive,Yes,"Slow, long lineup",0,Product: Functioning,On-demand pickup location
5,2608,Female,S,0,32.02,Active,No,Customer is important for the enjoyment of the...,0,Product: Functioning,Voucher
7,3382,Male,M,1,52.15,Inactive,No,They should upgrade me every time.,0,Service: Knowledge,Free Upgrade


Analyze the recommended actions for inputs from the test set.

**Note:** Execution of this cell could take several minutes.

In [137]:
results = []
prompt = Prompt(access_token, project_id)
comments = list(unsatisfied_test_data.Customer_Service)
offer_recommended = list(unsatisfied_test_data.Action.astype(str))

for input_text in comments:
    #print(input_text)
    input_text = input_text + '\n'
    res = prompt.generate("".join([offer_recommendation_instruction, " ", input_text, "Best Offer: "]), model_id, parameters).replace("\n", "")
    print(res)
    results.append(res)

On-demand pickup location
Free Upgrade
Premium features
Premium features
Premium features
Premium features
Voucher
Free Upgrade
Premium features
Voucher
Premium features
Premium features
Voucher
Premium features
Premium features
Premium features
On-demand pickup location
Premium features


In [138]:
results

['On-demand pickup location',
 'Free Upgrade',
 'Premium features',
 'Premium features',
 'Premium features',
 'Premium features',
 'Voucher',
 'Free Upgrade',
 'Premium features',
 'Voucher',
 'Premium features',
 'Premium features',
 'Voucher',
 'Premium features',
 'Premium features',
 'Premium features',
 'On-demand pickup location',
 'Premium features']

### Calculate the F1 micro score

In [140]:
from sklearn.metrics import f1_score

print('Offer recommendation f1_micro_score', f1_score(offer_recommended, results, average='micro'))

Offer recommendation f1_micro_score 0.4444444444444444


---

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.